In [ ]:
#with command line
/local2/home/tong/caffe-master/build/tools/caffe test -model ./fashion_train_val3.prototxt -weights ./fashion_predict3_15260.caffemodel -gpu 0 -iterations 500


In [1]:
#with python
import numpy as np
import sys
sys.path.insert(0,'/home/tonghe2/caffe-master/python')

In [2]:
import caffe

In [4]:
caffe.set_mode_gpu()
caffe.set_device(0)
net = caffe.Net('./fashion_deploy.prototxt',
                './fashion_predict.caffemodel',
                caffe.TEST)

In [3]:
image_root = '/home/tonghe2/Polyvore_small/Images/'
lines = open('/home/tonghe2/Polyvore_small/Exp1c/tuple_180/train_lmdb_list.txt').readlines()

In [4]:
print(len(lines))

1620000


In [7]:
import Image

size = 224, 224

blob = caffe.proto.caffe_pb2.BlobProto()
data = open('../VGG_mean.binaryproto', 'rb' ).read()
blob.ParseFromString(data)
VGG_mean = np.array( caffe.io.blobproto_to_array(blob) )
VGG_mean.astype(np.uint8)

data_posi = np.zeros((1,9,size[0],size[1]),np.uint8)
data_nega = np.zeros((1,9,size[0],size[1]),np.uint8)
count = 1

correct = 0
wrong = 0

print(VGG_mean[0,0,18,92])

113.369308472


In [39]:
for i in range(0,len(lines)):
    #image path posi & neg
    top_posi_img = image_root + lines[i+0].strip(' 1\r\n')
    bot_posi_img = image_root + lines[i+1].strip(' 1\r\n')
    sho_posi_img = image_root + lines[i+2].strip(' 1\r\n')
    top_nega_img = image_root + lines[i+3].strip(' 1\r\n')
    bot_nega_img = image_root + lines[i+4].strip(' 1\r\n')
    sho_nega_img = image_root + lines[i+5].strip(' 1\r\n')
    #read images posi & neg
    im_top_posi = Image.open(top_posi_img)
    im_bot_posi = Image.open(bot_posi_img)
    im_sho_posi = Image.open(sho_posi_img)
    im_top_nega = Image.open(top_nega_img)
    im_bot_nega = Image.open(bot_nega_img)
    im_sho_nega = Image.open(sho_nega_img)
    #resize to the same size posi & neg
    im_top_posi = im_top_posi.resize(size, Image.ANTIALIAS)
    im_bot_posi = im_bot_posi.resize(size, Image.ANTIALIAS)
    im_sho_posi = im_sho_posi.resize(size, Image.ANTIALIAS)
    im_top_nega = im_top_nega.resize(size, Image.ANTIALIAS)
    im_bot_nega = im_bot_nega.resize(size, Image.ANTIALIAS)
    im_sho_nega = im_sho_nega.resize(size, Image.ANTIALIAS)
    #image to nparray
    im_top_posi = np.array(im_top_posi)
    im_bot_posi = np.array(im_bot_posi)
    im_sho_posi = np.array(im_sho_posi)
    im_top_nega = np.array(im_top_nega)
    im_bot_nega = np.array(im_bot_nega)
    im_sho_nega = np.array(im_sho_nega)
    #change type to np.int64, np.uint8 (lmdb)
    im_top_posi.astype(np.uint8)
    im_bot_posi.astype(np.uint8)
    im_sho_posi.astype(np.uint8)
    im_top_nega.astype(np.uint8)
    im_bot_nega.astype(np.uint8)
    im_sho_nega.astype(np.uint8)
    #from (height,width,channel) to (channel,height,width)
    im_top_posi = np.rollaxis(im_top_posi,2,0)
    im_bot_posi = np.rollaxis(im_bot_posi,2,0)
    im_sho_posi = np.rollaxis(im_sho_posi,2,0)
    im_top_nega = np.rollaxis(im_top_nega,2,0)
    im_bot_nega = np.rollaxis(im_bot_nega,2,0)
    im_sho_nega = np.rollaxis(im_sho_nega,2,0)
    #from RGB to BGR
    im_top_posi = np.array([im_top_posi[2,],im_top_posi[1,],im_top_posi[0,]])
    im_bot_posi = np.array([im_bot_posi[2,],im_bot_posi[1,],im_bot_posi[0,]])
    im_sho_posi = np.array([im_sho_posi[2,],im_sho_posi[1,],im_sho_posi[0,]]) 
    im_top_nega = np.array([im_top_nega[2,],im_top_nega[1,],im_top_nega[0,]])
    im_bot_nega = np.array([im_bot_nega[2,],im_bot_nega[1,],im_bot_nega[0,]])
    im_sho_nega = np.array([im_sho_nega[2,],im_sho_nega[1,],im_sho_nega[0,]])   
    #zero mean normalization
    im_top_posi = im_top_posi - VGG_mean[0,:,:,:]
    im_bot_posi = im_bot_posi - VGG_mean[0,:,:,:]
    im_sho_posi = im_sho_posi - VGG_mean[0,:,:,:]
    im_top_nega = im_top_nega - VGG_mean[0,:,:,:]
    im_bot_nega = im_bot_nega - VGG_mean[0,:,:,:]
    im_sho_nega = im_sho_nega - VGG_mean[0,:,:,:]
    #stack over "channel"
    data_posi[count-1,0:3,:,:] = im_top_posi
    data_posi[count-1,3:6,:,:] = im_bot_posi
    data_posi[count-1,6:9,:,:] = im_sho_posi
    data_nega[count-1,0:3,:,:] = im_top_nega
    data_nega[count-1,3:6,:,:] = im_bot_nega
    data_nega[count-1,6:9,:,:] = im_sho_nega
    #predic posi outfit
    net.blobs['data'].data[...] = data_posi
    out = net.forward()
    posi_score = np.array(out['score'][0])
    #predic nega outfit
    net.blobs['data'].data[...] = data_nega
    out = net.forward()
    nega_score = np.array(out['score'][0])
    # correct or wrong
    if (posi_score > nega_score):
        correct = correct + 1
        print("Correct: {} > {}\n".format(posi_score,nega_score))
    else:
        wrong = wrong + 1
        print("Wrong: {} <= {}\n".format(posi_score,nega_score))
    i = i + 6
    
print("Total Correct: {}\n".format(correct))
print("Total Wrong: {}\n".format(wrong))

Wrong: [ 3892.60717773] <= [ 5347.86621094]

Wrong: [ 3834.07006836] <= [ 6165.37744141]

Wrong: [ 4717.98046875] <= [ 5043.37988281]

Correct: [ 5347.86621094] > [ 4884.27148438]

Correct: [ 6165.37744141] > [ 4970.4296875]

Correct: [ 5043.37988281] > [ 3914.3840332]

Correct: [ 4884.27148438] > [ 4033.09716797]

Correct: [ 4970.4296875] > [ 4556.828125]

Wrong: [ 3914.3840332] <= [ 4946.14501953]

Wrong: [ 4033.09716797] <= [ 4782.42871094]

Correct: [ 4556.828125] > [ 4346.34228516]

Correct: [ 4946.14501953] > [ 3995.32836914]

Correct: [ 4782.42871094] > [ 3476.96459961]

Correct: [ 4346.34228516] > [ 4070.74389648]

Wrong: [ 3995.32836914] <= [ 4421.12792969]

Wrong: [ 3476.96459961] <= [ 3737.52929688]

Correct: [ 4070.74389648] > [ 3436.77294922]

Correct: [ 4421.12792969] > [ 3822.12304688]

Wrong: [ 3737.52929688] <= [ 4382.29052734]

Wrong: [ 3436.77294922] <= [ 4796.04931641]

Wrong: [ 3822.12304688] <= [ 4309.24511719]

Correct: [ 4382.29052734] > [ 3736.29760742]

Correc

KeyboardInterrupt: 

In [42]:
net.params['user_top'][0].data.shape

OverflowError: long int too large to convert to int

In [26]:
#intermidiate ouput of fashionNet wrt posi outfit
#[(k, v.data.shape) for k, v in net.blobs.items()]
#[(k, v[0].data.shape) for k, v in net.params.items()]

In [27]:
#save caffemodel (not enough memeory on my laptop, but ok on ares)
net.save('fashion_predict.caffemodel')
